## Medici&oacute;n y An&aacute;lisis de Reacciones Emocionales a Trav&eacute;s de Estimulos Visuales

<style>
    .imagen-ajustada {
        width: 900px;
        height: 280px;
        border-radius: 20px;
    }
</style>

<img src="Emociones.jpg" alt="Una imagen" class="imagen-ajustada">

Las emociones son algo muy b&aacute;sico en el ser humano, sin embargo muchas veces no es necesario expresar la emoci&oacute;n por medio de palabras, sino que con el rostro se puede denotar lo que estas sintiendo. Basandonos en este concepto el proyecto fue planificado para ver si por medio de estimulos visuales, en este caso videos, se pueden obtener reacciones emocionales de acuerdo a lo que se le muestre.

Para el estudio se utilizaron 12 sujetos de prueba, en este caso fueron 8 hombres y 4 mujeres, con un rango de edad de 22 a 56 años con el fin de ver si generaba algun tipo de reaccion emocional por su parte.

El estudio consistia en mostrar 7 videos de 30 segundos conformados de la siguiente manera:
- Un video de relajacion, con el fin de que el sujeto este en calma.
- Un video donde una nina juega con su perro a que es veterinaria, con el fin de tratar de obtener una reaccion de felicidad o gracia. 
- Un video de relajacion, con el fin de que el sujeto vuelva a un estado neutral.
- Un video donde un hombre golpea a su mujer con un bebe en brazos, con el fin de tratar de obtener una reaccion de enojo, impotencia y rabia.
- Un video de relajacion, con el fin de que el sujeto vuelva a un estado neutral.
- Un video donde una mujer esta abrazando a su perro mientras proceden a dormirlo, con el fin de obtener una reaccion de tristeza.
- Finalmente, un video de relajacion con el fin de que el sujeto vuelva a un estado neutral.      

El fin de comenzar y terminar con un video de relajacion es para ver si hay algun cambio entre el estado inicial y final, como nos indican en la materia de analisis y diseno de experimentos en ciencias de la computacion, tenemos que tener un pre y un post de en el estudio.

## Librerias Importadas

In [1]:
import os
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
from pydub import AudioSegment
import subprocess
from moviepy.editor import VideoFileClip
import pandas as pd

## Procesamiento de datos

Para poder comenzar con el tratamiento de los datos fue importante separar los videos de los audios, por lo cual se uso un script para separarlos en distintas carpetas. Despues de eso se trato de cortar los audios a una misma cantidad de tiempo como en el siguiente codigo.

In [21]:
# Ruta a la carpeta que contiene tus archivos de audio
carpeta_audios = './Audios/'

# Ruta a la carpeta donde deseas guardar los audios recortados
carpeta_destino = './AudiosRecortados/'

# Crear la carpeta de destino si no existe
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Iterar sobre cada archivo en la carpeta
for archivo in os.listdir(carpeta_audios):
    ruta_archivo = os.path.join(carpeta_audios, archivo)
    
    # Cargar el archivo de audio
    audio = AudioSegment.from_file(ruta_archivo)
    
    # Descartar los primeros 30 segundos del audio
    audio_recortado = audio[40000:]  # 30 segundos = 30,000 milisegundos
    
    # Guardar el audio recortado
    ruta_destino = os.path.join(carpeta_destino, archivo)
    audio_recortado.export(ruta_destino, format="wav")  # Cambia "wav" si necesitas otro formato

print("Recorte completado.")

Recorte completado.


Sin embargo, no se tomo en cuenta que no todos los audios duraban la misma cantidad de tiempo, asi como las personas hablaban mas o hablaban menos, por lo cual se opto por un script para cortar de forma personalizada cada audio y asi obtener de forma precisa los comentarios de los sujetos de prueba.

In [30]:
# Ruta al archivo de audio específico
ruta_audio = './Audios/SUJETO6_TRISTEZA.wav'

# Puntos de inicio y fin del segmento en milisegundos
# Por ejemplo, para comenzar en el segundo 20 y terminar en el segundo 60
inicio_ms = 30000  # Inicio a los 20 segundos
duracion_ms = 17000  # Duración de 40 segundos desde el punto de inicio

# Cargar el archivo de audio
audio = AudioSegment.from_file(ruta_audio)

# Recortar el segmento deseado
segmento_audio = audio[inicio_ms:inicio_ms+duracion_ms]

# Ruta para guardar el segmento recortado
ruta_destino = './AudiosRecortados/SUJETO6_TRISTEZA_R.wav'

# Guardar el segmento recortado
segmento_audio.export(ruta_destino, format="wav")

print("Segmento recortado y guardado.")


Segmento recortado y guardado.


Para el siguiente caso fue recortar los videos unicamente cuando se tiene la reaccion del sujeto con el video, por lo cual se utilizo un script para cortar los primeros segundos en base a la duracion del video.

In [45]:
# Define la ruta de la carpeta que contiene los videos originales
carpeta_videos = './Videos/'

# Define la ruta de la carpeta donde guardarás los videos recortados
carpeta_destino = './VideosRecortados/'

# Crea la carpeta de destino si no existe
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Itera sobre cada archivo en la carpeta de videos
for archivo in os.listdir(carpeta_videos):
    # Verifica si el archivo termina en '_N1.MOV'
    if archivo.endswith('_N4.MOV'):
        ruta_completa = os.path.join(carpeta_videos, archivo)
        
        # Carga el video
        video = VideoFileClip(ruta_completa)
        
        # Recorta el video desde el segundo 0 hasta el segundo 31
        video_recortado = video.subclip(0, 33)
        
        # Define la ruta del archivo de destino
        ruta_destino = os.path.join(carpeta_destino, archivo)
        
        # Guarda el video recortado en la carpeta de destino
        video_recortado.write_videofile(ruta_destino, codec="libx264", audio_codec="aac")

        # Cierra el clip para liberar recursos
        video.close()
        video_recortado.close()

print("Todos los videos han sido recortados y guardados.")


Moviepy - Building video ./VideosRecortados/SUJETO10_N4.MOV.
MoviePy - Writing audio in SUJETO10_N4TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./VideosRecortados/SUJETO10_N4.MOV



Moviepy - Done !
Moviepy - video ready ./VideosRecortados/SUJETO10_N4.MOV
Moviepy - Building video ./VideosRecortados/SUJETO11_N4.MOV.
MoviePy - Writing audio in SUJETO11_N4TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./VideosRecortados/SUJETO11_N4.MOV



Moviepy - Done !
Moviepy - video ready ./VideosRecortados/SUJETO11_N4.MOV
Moviepy - Building video ./VideosRecortados/SUJETO12_N4.MOV.
MoviePy - Writing audio in SUJETO12_N4TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready ./VideosRecortados/SUJETO7_N4.MOV
Moviepy - Building video ./VideosRecortados/SUJETO8_N4.MOV.
MoviePy - Writing audio in SUJETO8_N4TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./VideosRecortados/SUJETO8_N4.MOV



Moviepy - Done !
Moviepy - video ready ./VideosRecortados/SUJETO8_N4.MOV
Moviepy - Building video ./VideosRecortados/SUJETO9_N4.MOV.
MoviePy - Writing audio in SUJETO9_N4TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./VideosRecortados/SUJETO9_N4.MOV



Moviepy - Done !
Moviepy - video ready ./VideosRecortados/SUJETO9_N4.MOV
Todos los videos han sido recortados y guardados.


En el caso del participante 9 el internet estaba fallando, por lo cual el video comenzo 8 segundos despues de comenzar a grabar, por lo cual individualmente se recorto el video con el siguiente codigo:

In [46]:
# Define la ruta completa al video que deseas recortar
ruta_video = './Videos/SUJETO9_N4.MOV'

# Define la ruta de la carpeta donde guardarás el video recortado
carpeta_destino = './VideosRecortados/'

# Crea la carpeta de destino si no existe
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Comprueba si el archivo termina en '_N1.MOV'
if ruta_video.endswith('_N4.MOV'):
    # Carga el video
    video = VideoFileClip(ruta_video)
    
    # Recorta el video desde el segundo 0 hasta el segundo 31
    video_recortado = video.subclip(7, 38)
    
    # Extrae el nombre del archivo del video original
    nombre_archivo = os.path.basename(ruta_video)
    
    # Define la ruta del archivo de destino
    ruta_destino = os.path.join(carpeta_destino, nombre_archivo)
    
    # Guarda el video recortado en la carpeta de destino
    video_recortado.write_videofile(ruta_destino, codec="libx264", audio_codec="aac")
    
    # Cierra el clip para liberar recursos
    video.close()
    video_recortado.close()

    print(f"El video {nombre_archivo} ha sido recortado y guardado.")
else:
    print("El archivo no termina en '_N3.MOV', por lo que no se ha recortado.")


Moviepy - Building video ./VideosRecortados/SUJETO9_N4.MOV.
MoviePy - Writing audio in SUJETO9_N4TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./VideosRecortados/SUJETO9_N4.MOV



Moviepy - Done !
Moviepy - video ready ./VideosRecortados/SUJETO9_N4.MOV
El video SUJETO9_N4.MOV ha sido recortado y guardado.


## An&aacute;lisis Exploratorio

No solo se busco ver que si se obtenian reacciones por medio de estimulos visuales, tambien se aplico un cuestionario donde los sujetos de prueba daban puntuacion dependiendo de cuanto sintieron la emocion del 1 al 5, donde 1 es poco y 5 es mucho.
Por lo cual, en base a sus auto reportes se puede ver por medio de la grafica que la mayoria 

## Pruebas

In [2]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Importa las utilidades de dibujo y los estilos de dibujo de MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

mp_face_mesh = mp.solutions.face_mesh

# Inicializa FaceMesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture('./Videos/SUJETO10_ENOJO.MOV')  # Reemplaza esto con la ruta a tu video

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Fin del video o error al cargar el archivo.")  # Finaliza el bucle si no se pudo leer el video
        break
    
    # Cambia la resolución de la imagen a 640x480
    image = cv2.resize(image, (720, 540))

    # Convertir la imagen de BGR a RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Detección de puntos de referencia faciales
    results = face_mesh.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

    cv2.imshow('Face Landmark Detection', image)
    if cv2.waitKey(1) & 0xFF == 27:  # Presiona ESC para salir
        break

cap.release()
cv2.destroyAllWindows()

## Extracci&oacute;n de Caracter&iacute;sticas 

In [25]:
path = './VideosSegmentados.csv'
segmentado_df = pd.read_csv(path)
print(segmentado_df.head())


                  SUJETO INICIO   FIN      LABEL
0      SUJETO1_ENOJO.MOV   0:03  0:14      ENOJO
1         SUJETO1_N1.MOV   0:10  0:20    NEUTRAL
2  SUJETO1_FELICIDAD.MOV   0:07  0:14  FELICIDAD
3   SUJETO1_TRISTEZA.MOV   0:03  0:13   TRISTEZA
4      SUJETO2_ENOJO.MOV   0:05  0:15      ENOJO


In [37]:
import pandas as pd
from moviepy.editor import VideoFileClip
import os

# Cambia esta variable por la ruta a tu archivo CSV
csv_file = './VideosSegmentados.csv'

# Cambia esta variable por la ruta a la carpeta donde están tus videos originales
videos_folder = './Videos/'

# Ruta a la carpeta donde quieres guardar los videos segmentados
segmented_videos_folder = './VideosSegmentados'

# Crea la carpeta para los videos segmentados si no existe
if not os.path.exists(segmented_videos_folder):
    os.makedirs(segmented_videos_folder)

df = pd.read_csv(csv_file)

# Itera sobre cada fila del DataFrame
for index, row in df.iterrows():
    video_path = os.path.join(videos_folder, row['SUJETO'])
    inicio = row['INICIO']
    fin = row['FIN']
    
    # Carga el video
    clip = VideoFileClip(video_path)
    
    # Convierte los tiempos de inicio y fin a segundos
    start_seconds = int(inicio.split(':')[0]) * 60 + int(inicio.split(':')[1])
    end_seconds = int(fin.split(':')[0]) * 60 + int(fin.split(':')[1])
    
    # Verifica que el tiempo de inicio y fin estén dentro de la duración del video
    if start_seconds >= clip.duration or end_seconds > clip.duration:
        print(f"Error en {video_path}: El tiempo de inicio o fin está fuera de la duración del video.")
        continue  # Salta este video y continúa con el siguiente

    # Segmenta el video
    new_clip = clip.subclip(start_seconds, end_seconds)
    
    # Segmenta el video
    new_clip = clip.subclip(start_seconds, end_seconds)
    
    # Define la ruta del nuevo video segmentado, incluyendo la carpeta VideosSegmentados
    new_video_filename = os.path.splitext(row['SUJETO'])[0] + '_segmentado.mp4'
    new_video_path = os.path.join(segmented_videos_folder, new_video_filename)
    
    # Escribe el video segmentado en el disco
    new_clip.write_videofile(new_video_path, codec='libx264', audio_codec='aac')

    # Libera recursos
    clip.close()
    new_clip.close()


Moviepy - Building video ./VideosSegmentados\SUJETO1_ENOJO_segmentado.mp4.
MoviePy - Writing audio in SUJETO1_ENOJO_segmentadoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./VideosSegmentados\SUJETO1_ENOJO_segmentado.mp4



Moviepy - Done !
Moviepy - video ready ./VideosSegmentados\SUJETO1_ENOJO_segmentado.mp4
Moviepy - Building video ./VideosSegmentados\SUJETO1_N1_segmentado.mp4.
MoviePy - Writing audio in SUJETO1_N1_segmentadoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./VideosSegmentados\SUJETO1_N1_segmentado.mp4



Moviepy - Done !
Moviepy - video ready ./VideosSegmentados\SUJETO1_N1_segmentado.mp4
Moviepy - Building video ./VideosSegmentados\SUJETO1_FELICIDAD_segmentado.mp4.
MoviePy - Writing audio in SUJETO1_FELICIDAD_segmentadoTEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready ./VideosSegmentados\SUJETO12_N1_segmentado.mp4
Moviepy - Building video ./VideosSegmentados\SUJETO12_FELICIDAD_segmentado.mp4.
MoviePy - Writing audio in SUJETO12_FELICIDAD_segmentadoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./VideosSegmentados\SUJETO12_FELICIDAD_segmentado.mp4



Moviepy - Done !
Moviepy - video ready ./VideosSegmentados\SUJETO12_FELICIDAD_segmentado.mp4
Moviepy - Building video ./VideosSegmentados\SUJETO12_TRISTEZA_segmentado.mp4.
MoviePy - Writing audio in SUJETO12_TRISTEZA_segmentadoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./VideosSegmentados\SUJETO12_TRISTEZA_segmentado.mp4



Moviepy - Done !
Moviepy - video ready ./VideosSegmentados\SUJETO12_TRISTEZA_segmentado.mp4


In [38]:
import cv2
import os

# Ruta a la carpeta que contiene los videos
videos_folder = './VideosSegmentados/'

# Ruta a la carpeta donde se guardarán las imágenes
images_folder = './Imagenes/'

# Crea la carpeta para las imágenes si no existe
if not os.path.exists(images_folder):
    os.makedirs(images_folder)

# Recorre todos los archivos en la carpeta de videos
for video_file in os.listdir(videos_folder):
    video_path = os.path.join(videos_folder, video_file)
    
    # Verifica si el archivo es un video abriéndolo con OpenCV
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"No se pudo abrir el video {video_file}")
        continue

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Extrae un frame cada 30 frames
        if frame_count % 30 == 0:
            frame_filename = f"{os.path.splitext(video_file)[0]}_frame{frame_count}.jpg"
            frame_path = os.path.join(images_folder, frame_filename)
            cv2.imwrite(frame_path, frame)
            print(f"Frame guardado: {frame_filename}")

        frame_count += 1

    # Libera el objeto cap y cierra todas las ventanas de OpenCV
    cap.release()

cv2.destroyAllWindows()


Frame guardado: SUJETO10_ENOJO_segmentado_frame0.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame30.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame60.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame90.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame120.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame150.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame180.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame210.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame240.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame270.jpg
Frame guardado: SUJETO10_ENOJO_segmentado_frame300.jpg
Frame guardado: SUJETO10_FELICIDAD_segmentado_frame0.jpg
Frame guardado: SUJETO10_FELICIDAD_segmentado_frame30.jpg
Frame guardado: SUJETO10_FELICIDAD_segmentado_frame60.jpg
Frame guardado: SUJETO10_FELICIDAD_segmentado_frame90.jpg
Frame guardado: SUJETO10_FELICIDAD_segmentado_frame120.jpg
Frame guardado: SUJETO10_FELICIDAD_segmentado_frame150.jpg
Frame guardado: SUJETO10_FELICIDAD_segmentado_frame

In [4]:
import cv2
import mediapipe as mp
import pandas as pd
import os

# Inicializa mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Configuración de FaceMesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Ruta a la carpeta de imágenes
images_folder = './Imagenes/'

# DataFrame para almacenar los keypoints
df_keypoints = pd.DataFrame()

# Recorre todas las imágenes en la carpeta
for image_file in os.listdir(images_folder):
    image_path = os.path.join(images_folder, image_file)
    
    # Lee la imagen
    image = cv2.imread(image_path)
    
    # Verifica si la imagen se cargó correctamente
    if image is None:
        print(f"No se pudo cargar la imagen {image_file}")
        continue

    # Cambia la resolución de la imagen a 720x540
    image = cv2.resize(image, (720, 540))

    # Convertir la imagen de BGR a RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Procesa la imagen para detectar los landmarks faciales
    results = face_mesh.process(image_rgb)

    # Si se encuentran landmarks faciales, extrae la información
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extrae los keypoints de cada landmark facial
            keypoints = []
            for landmark in face_landmarks.landmark:
                keypoints.append({
                    'x': landmark.x,
                    'y': landmark.y,
                    'z': landmark.z
                })
            # Añade los keypoints al DataFrame
            df_image_keypoints = pd.DataFrame(keypoints)
            df_image_keypoints['image'] = image_file
            df_keypoints = pd.concat([df_keypoints, df_image_keypoints], ignore_index=True)

# Cierra FaceMesh
face_mesh.close()


In [5]:
df_keypoints

,x,y,z,image
0,0.514819,0.862191,-0.031919,SUJETO10_ENOJO_segmentado_frame0.jpg
1,0.521086,0.776935,-0.065382,SUJETO10_ENOJO_segmentado_frame0.jpg
2,0.518349,0.793673,-0.033238,SUJETO10_ENOJO_segmentado_frame0.jpg
3,0.517264,0.708311,-0.052481,SUJETO10_ENOJO_segmentado_frame0.jpg
4,0.523252,0.757754,-0.070094,SUJETO10_ENOJO_segmentado_frame0.jpg
...,...,...,...,...
223699,0.541657,0.608421,0.009365,SUJETO9_TRISTEZA_segmentado_frame90.jpg
223700,0.554624,0.611048,0.009365,SUJETO9_TRISTEZA_segmentado_frame90.jpg
223701,0.543330,0.592531,0.009365,SUJETO9_TRISTEZA_segmentado_frame90.jpg
223702,0.528629,0.605639,0.009365,SUJETO9_TRISTEZA_segmentado_frame90.jpg


In [6]:
# Guarda el DataFrame en un archivo CSV si es necesario
df_keypoints.to_csv('facemesh_keypoints.csv', index=False)

In [8]:
import cv2
import mediapipe as mp
import pandas as pd
import os

# Inicializa mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Configuración de FaceMesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Ruta a la carpeta de imágenes
images_folder = './Imagenes'

# DataFrame para almacenar los keypoints
df_keypoints = pd.DataFrame()

# Recorre todas las imágenes en la carpeta
for image_file in os.listdir(images_folder):
    image_path = os.path.join(images_folder, image_file)
    
    # Lee la imagen
    image = cv2.imread(image_path)
    
    # Verifica si la imagen se cargó correctamente
    if image is None:
        print(f"No se pudo cargar la imagen {image_file}")
        continue

    # Cambia la resolución de la imagen a 720x540
    image = cv2.resize(image, (720, 540))

    # Convertir la imagen de BGR a RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Procesa la imagen para detectar los landmarks faciales
    results = face_mesh.process(image_rgb)

    # Si se encuentran landmarks faciales, extrae la información
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Aplanar los keypoints para una sola fila
            keypoints = {'label': 'inserta_tu_label_aquí'}
            for i, landmark in enumerate(face_landmarks.landmark):
                keypoints[f'keypointX_{i}'] = landmark.x
                keypoints[f'keypointY_{i}'] = landmark.y

            # Añade los keypoints aplanados al DataFrame
            df_keypoints = df_keypoints.append(keypoints, ignore_index=True)

# Guarda el DataFrame en un archivo CSV si es necesario
#df_keypoints.to_csv('facemesh_keypoints_flattened.csv', index=False)

# Cierra FaceMesh
face_mesh.close()


AttributeError: 'DataFrame' object has no attribute 'append'

In [18]:
import cv2
import mediapipe as mp
import pandas as pd
import os

# Definir la función para extraer landmarks faciales de un fotograma
def extract_landmarks(frame):
    # Inicializar MediaPipe Face Mesh cada vez que se llama a la función
    with mp.solutions.face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        min_detection_confidence=0.5) as face_mesh:

        # Convertir la imagen a RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen y obtener los resultados de los landmarks faciales
        results = face_mesh.process(image_rgb)
        
        # Verificar si se detectaron landmarks faciales
        if results.multi_face_landmarks:
            # Obtener los landmarks faciales del primer rostro detectado
            face_landmarks = results.multi_face_landmarks[0]
            
            # Convertir los landmarks a una lista de tuplas de coordenadas (x, y, z)
            landmarks = [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]
            
            return landmarks  # Devolver la lista de landmarks faciales

    return None

path = "./Videos/"
list_dir = os.listdir(path)
df_all_data_landmarks = pd.DataFrame(columns=['Video', 'Frame', 'Landmarks'])  # Asegúrate de usar el nombre correcto de DataFrame aquí
landmarks_filter_data = pd.DataFrame(columns=['filename'] + [f"x_{i}" for i in range(468)] + [f"y_{i}" for i in range(468)] + [f"z_{i}" for i in range(468)] + ['label'])

for file in list_dir:
    cap = cv2.VideoCapture(path + file)
    if not cap.isOpened():
        print(f"Error al abrir el video: {file}")
        continue

    while True:
        success, frame = cap.read()
        if not success:
            break

        landmarks = extract_landmarks(frame)
        
        if landmarks:
            landmark_dict = {"filename": file}
            for i, lm in enumerate(landmarks):
                landmark_dict[f"x_{i}"] = lm[0]
                landmark_dict[f"y_{i}"] = lm[1]
                landmark_dict[f"z_{i}"] = lm[2]

            label = file.split('_')[1]
            landmark_dict['label'] = label

            # Crear un DataFrame temporal con los datos de esta fila
            temp_df = pd.DataFrame([landmark_dict])

            # Concatenar el DataFrame temporal con el DataFrame principal
            landmarks_filter_data = pd.concat([landmarks_filter_data, temp_df], ignore_index=True)

    cap.release()


KeyboardInterrupt: 

In [12]:
df_all_data_landmarks = landmarks_data
# Guardar el DataFrame con los landmarks de todos los videos como un archivo CSV
df_all_data_landmarks.to_csv('df_all_data_landmarks.csv', index=False)

In [19]:


# Inicializa MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Lista para almacenar los datos de los landmarks
landmarks_data = []

# Carpeta que contiene los videos
videos_folder = './Videos/'

# Lista todos los archivos en la carpeta que terminan con _N1.MOV
video_files = [f for f in os.listdir(videos_folder) if f.endswith('_N1.MOV')]

# Procesa cada video que cumple con el criterio
for video_file in video_files:
    video_path = os.path.join(videos_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break  # Fin del video

            # Preprocesa la imagen
            image = cv2.resize(image, (720, 540))
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)

            # Extrae landmarks faciales si están disponibles
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Extrae landmarks y los almacena en la lista
                    for idx, landmark in enumerate(face_landmarks.landmark):
                        landmarks_data.append({
                            'Video': video_file,
                            'Landmark_ID': idx,
                            'x': landmark.x,
                            'y': landmark.y,
                            'z': landmark.z
                        })

    cap.release()

# Crea un DataFrame a partir de la lista de datos
df_all_videos_landmarks = pd.DataFrame(landmarks_data)

cv2.destroyAllWindows()

# Muestra los primeros registros del DataFrame
print(df_all_videos_landmarks.head())

# Opcional: Guarda el DataFrame a un archivo CSV
# df_all_videos_landmarks.to_csv('video_landmarks_N1.csv', index=False)


             Video  Landmark_ID         x         y         z
0  SUJETO10_N1.MOV            0  0.535982  0.731780 -0.037235
1  SUJETO10_N1.MOV            1  0.538623  0.643297 -0.072559
2  SUJETO10_N1.MOV            2  0.535853  0.665372 -0.038538
3  SUJETO10_N1.MOV            3  0.532896  0.566527 -0.055616
4  SUJETO10_N1.MOV            4  0.540620  0.620441 -0.077139


In [7]:
df

,K1X,K2X,K3X,K4X,K5X,K6X,K7X,K8X,K9X,K10X,...,K459Y,K460Y,K461Y,K462Y,K463Y,K464Y,K465Y,K466Y,K467Y,K468Y


In [13]:
import cv2
import mediapipe as mp
import pandas as pd
import os

# Inicializa MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh

# Lista para almacenar los datos de los landmarks
landmarks_data = []

# Carpeta que contiene las imágenes
images_folder = './Imagenes'

# Inicializa FaceMesh con las opciones deseadas
with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:

    # Lista todos los archivos en la carpeta de imágenes
    image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg') or f.endswith('.png')]

    # Procesa cada imagen que cumple con el criterio
    for image_file in image_files:
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)
        
        # Verifica si la imagen se cargó correctamente
        if image is None:
            print(f"No se pudo cargar la imagen {image_file}")
            continue
        
        # Cambia la resolución de la imagen y la convierte a RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Procesa la imagen para obtener los landmarks
        results = face_mesh.process(image_rgb)
        
        # Extrae landmarks faciales si están disponibles
        if results.multi_face_landmarks:
            # Cada imagen dará lugar a una sola fila con todos los landmarks
            row = {'Image': image_file}
            for landmark_id, landmark in enumerate(results.multi_face_landmarks[0].landmark):
                # Añade las coordenadas x, y de cada landmark al diccionario de la fila
                row[f'K{landmark_id + 1}X'] = landmark.x
                row[f'K{landmark_id + 1}Y'] = landmark.y
            
            landmarks_data.append(row)

# Crea un DataFrame a partir de la lista de datos
df_flat_landmarks = pd.DataFrame(landmarks_data)

# Muestra los primeros registros del DataFrame
print(df_flat_landmarks.head())

# Opcional: Guarda el DataFrame a un archivo CSV
# df_flat_landmarks.to_csv('flat_landmarks_images.csv', index=False)


                                    Image       K1X       K1Y       K2X  \
0    SUJETO10_ENOJO_segmentado_frame0.jpg  0.516717  0.862611  0.525122   
1  SUJETO10_ENOJO_segmentado_frame120.jpg  0.522165  0.873315  0.534889   
2  SUJETO10_ENOJO_segmentado_frame150.jpg  0.521257  0.869050  0.532735   
3  SUJETO10_ENOJO_segmentado_frame180.jpg  0.519951  0.869113  0.532665   
4  SUJETO10_ENOJO_segmentado_frame210.jpg  0.522861  0.869178  0.534696   

        K2Y       K3X       K3Y       K4X       K4Y       K5X  ...     K474X  \
0  0.781059  0.522194  0.800093  0.520307  0.707113  0.527029  ...  0.586153   
1  0.798453  0.530828  0.816012  0.530545  0.722064  0.537171  ...  0.594279   
2  0.793808  0.528710  0.811547  0.528189  0.717880  0.534973  ...  0.593558   
3  0.793192  0.528782  0.810866  0.527933  0.718591  0.534852  ...  0.592393   
4  0.794747  0.530758  0.812396  0.530206  0.719392  0.536961  ...  0.594907   

      K474Y     K475X     K475Y     K476X     K476Y     K477X     K4

In [14]:
df_flat_landmarks

,Image,K1X,K1Y,K2X,K2Y,K3X,K3Y,K4X,K4Y,K5X,...,K474X,K474Y,K475X,K475Y,K476X,K476Y,K477X,K477Y,K478X,K478Y
0,SUJETO10_ENOJO_segmentado_frame0.jpg,0.516717,0.862611,0.525122,0.781059,0.522194,0.800093,0.520307,0.707113,0.527029,...,0.586153,0.654642,0.597166,0.657380,0.587159,0.638631,0.575269,0.651748,0.585243,0.670582
1,SUJETO10_ENOJO_segmentado_frame120.jpg,0.522165,0.873315,0.534889,0.798453,0.530828,0.816012,0.530545,0.722064,0.537171,...,0.594279,0.667019,0.605121,0.669919,0.595619,0.650519,0.583513,0.663937,0.593000,0.683557
2,SUJETO10_ENOJO_segmentado_frame150.jpg,0.521257,0.869050,0.532735,0.793808,0.528710,0.811547,0.528189,0.717880,0.534973,...,0.593558,0.667484,0.604407,0.670765,0.594810,0.650166,0.582683,0.663929,0.592320,0.684695
3,SUJETO10_ENOJO_segmentado_frame180.jpg,0.519951,0.869113,0.532665,0.793192,0.528782,0.810866,0.527933,0.718591,0.534852,...,0.592393,0.671306,0.602347,0.674140,0.593610,0.656219,0.582534,0.668459,0.591239,0.686257
4,SUJETO10_ENOJO_segmentado_frame210.jpg,0.522861,0.869178,0.534696,0.794747,0.530758,0.812396,0.530206,0.719392,0.536961,...,0.594907,0.669738,0.605531,0.672815,0.596163,0.653849,0.584328,0.666554,0.593704,0.685507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,SUJETO9_TRISTEZA_segmentado_frame240.jpg,0.453008,0.793498,0.458177,0.714694,0.456633,0.737436,0.452061,0.642798,0.459526,...,0.525427,0.602941,0.537411,0.605893,0.526688,0.585254,0.513452,0.599915,0.524098,0.620761
465,SUJETO9_TRISTEZA_segmentado_frame270.jpg,0.455805,0.793401,0.464809,0.713732,0.463130,0.735859,0.458704,0.641522,0.466215,...,0.532660,0.605918,0.544063,0.608902,0.533972,0.588991,0.521302,0.602830,0.531304,0.622859
466,SUJETO9_TRISTEZA_segmentado_frame30.jpg,0.451938,0.797064,0.454748,0.717145,0.453761,0.739351,0.448418,0.644463,0.455899,...,0.523657,0.601855,0.535964,0.604622,0.524808,0.583856,0.511326,0.598940,0.522452,0.619921
467,SUJETO9_TRISTEZA_segmentado_frame60.jpg,0.463313,0.787222,0.469396,0.711167,0.467647,0.732886,0.462881,0.640404,0.470740,...,0.535791,0.601218,0.547727,0.604323,0.537343,0.584747,0.523882,0.598001,0.534184,0.617750


In [18]:
import cv2
import mediapipe as mp
import pandas as pd
import os

# Inicializa MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh

# Lista para almacenar los datos de los landmarks
landmarks_data = []

# Carpeta que contiene las imágenes
images_folder = './Imagenes/'

# Inicializa FaceMesh con las opciones deseadas
with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:

    # Lista todos los archivos en la carpeta de imágenes
    image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg') or f.endswith('.png')]

    # Procesa cada imagen que cumple con el criterio
    for image_file in image_files:
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)
        
        # Verifica si la imagen se cargó correctamente
        if image is None:
            print(f"No se pudo cargar la imagen {image_file}")
            continue
        
        # Cambia la resolución de la imagen y la convierte a RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Procesa la imagen para obtener los landmarks
        results = face_mesh.process(image_rgb)
        
        # Extrae landmarks faciales si están disponibles
        if results.multi_face_landmarks:
            # Cada imagen dará lugar a una sola fila con todos los landmarks
            row = {'Image': image_file}
            for landmark_id, landmark in enumerate(results.multi_face_landmarks[0].landmark):
                # Añade las coordenadas x, y de cada landmark al diccionario de la fila
                row[f'K{landmark_id + 1}X'] = landmark.x
                row[f'K{landmark_id + 1}Y'] = landmark.y
            
            # Extrae el label del nombre del archivo
            label = image_file.split('_')[1]  # Esto toma el elemento entre el primer y segundo '_'
            if label == 'N1':  # Cambia 'N1' a 'NEUTRAL'
                label = 'NEUTRAL'
            row['Label'] = label
            
            landmarks_data.append(row)

# Crea un DataFrame a partir de la lista de datos
df_flat_landmarks = pd.DataFrame(landmarks_data)

# Muestra los primeros registros del DataFrame
print(df_flat_landmarks.head())

# Opcional: Guarda el DataFrame a un archivo CSV
# df_flat_landmarks.to_csv('flat_landmarks_images_with_label.csv', index=False)


                                    Image       K1X       K1Y       K2X  \
0    SUJETO10_ENOJO_segmentado_frame0.jpg  0.516717  0.862611  0.525122   
1  SUJETO10_ENOJO_segmentado_frame120.jpg  0.522165  0.873315  0.534889   
2  SUJETO10_ENOJO_segmentado_frame150.jpg  0.521257  0.869050  0.532735   
3  SUJETO10_ENOJO_segmentado_frame180.jpg  0.519951  0.869113  0.532665   
4  SUJETO10_ENOJO_segmentado_frame210.jpg  0.522861  0.869178  0.534696   

        K2Y       K3X       K3Y       K4X       K4Y       K5X  ...     K474Y  \
0  0.781059  0.522194  0.800093  0.520307  0.707113  0.527029  ...  0.654642   
1  0.798453  0.530828  0.816012  0.530545  0.722064  0.537171  ...  0.667019   
2  0.793808  0.528710  0.811547  0.528189  0.717880  0.534973  ...  0.667484   
3  0.793192  0.528782  0.810866  0.527933  0.718591  0.534852  ...  0.671306   
4  0.794747  0.530758  0.812396  0.530206  0.719392  0.536961  ...  0.669738   

      K475X     K475Y     K476X     K476Y     K477X     K477Y     K4

In [19]:
df_flat_landmarks

,Image,K1X,K1Y,K2X,K2Y,K3X,K3Y,K4X,K4Y,K5X,...,K474Y,K475X,K475Y,K476X,K476Y,K477X,K477Y,K478X,K478Y,Label
0,SUJETO10_ENOJO_segmentado_frame0.jpg,0.516717,0.862611,0.525122,0.781059,0.522194,0.800093,0.520307,0.707113,0.527029,...,0.654642,0.597166,0.657380,0.587159,0.638631,0.575269,0.651748,0.585243,0.670582,ENOJO
1,SUJETO10_ENOJO_segmentado_frame120.jpg,0.522165,0.873315,0.534889,0.798453,0.530828,0.816012,0.530545,0.722064,0.537171,...,0.667019,0.605121,0.669919,0.595619,0.650519,0.583513,0.663937,0.593000,0.683557,ENOJO
2,SUJETO10_ENOJO_segmentado_frame150.jpg,0.521257,0.869050,0.532735,0.793808,0.528710,0.811547,0.528189,0.717880,0.534973,...,0.667484,0.604407,0.670765,0.594810,0.650166,0.582683,0.663929,0.592320,0.684695,ENOJO
3,SUJETO10_ENOJO_segmentado_frame180.jpg,0.519951,0.869113,0.532665,0.793192,0.528782,0.810866,0.527933,0.718591,0.534852,...,0.671306,0.602347,0.674140,0.593610,0.656219,0.582534,0.668459,0.591239,0.686257,ENOJO
4,SUJETO10_ENOJO_segmentado_frame210.jpg,0.522861,0.869178,0.534696,0.794747,0.530758,0.812396,0.530206,0.719392,0.536961,...,0.669738,0.605531,0.672815,0.596163,0.653849,0.584328,0.666554,0.593704,0.685507,ENOJO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,SUJETO9_TRISTEZA_segmentado_frame240.jpg,0.453008,0.793498,0.458177,0.714694,0.456633,0.737436,0.452061,0.642798,0.459526,...,0.602941,0.537411,0.605893,0.526688,0.585254,0.513452,0.599915,0.524098,0.620761,TRISTEZA
465,SUJETO9_TRISTEZA_segmentado_frame270.jpg,0.455805,0.793401,0.464809,0.713732,0.463130,0.735859,0.458704,0.641522,0.466215,...,0.605918,0.544063,0.608902,0.533972,0.588991,0.521302,0.602830,0.531304,0.622859,TRISTEZA
466,SUJETO9_TRISTEZA_segmentado_frame30.jpg,0.451938,0.797064,0.454748,0.717145,0.453761,0.739351,0.448418,0.644463,0.455899,...,0.601855,0.535964,0.604622,0.524808,0.583856,0.511326,0.598940,0.522452,0.619921,TRISTEZA
467,SUJETO9_TRISTEZA_segmentado_frame60.jpg,0.463313,0.787222,0.469396,0.711167,0.467647,0.732886,0.462881,0.640404,0.470740,...,0.601218,0.547727,0.604323,0.537343,0.584747,0.523882,0.598001,0.534184,0.617750,TRISTEZA


In [20]:
#Opcional: Guarda el DataFrame a un archivo CSV
df_flat_landmarks.to_csv('flat_landmarks_images_with_label.csv', index=False)

## Modelos a Evaluar

## Evaluando los modelos

## Seleccionando el mejor modelo

## Validaci&oacute;n

## Conclusiones